# Table S06. Heritability estimates for toxicant conditions

In [ ]:
library(dplyr)



Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'data.table'

The following objects are masked from 'package:dplyr':

    between, first, last

In [ ]:

# load the functions to save the output
source("bin/outs.R")



Attaching package: 'kableExtra'

The following objects are masked from 'package:flextable':

    as_image, footnote

The following object is masked from 'package:dplyr':

    group_rows

here() starts at /Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript

$strain_table
$strain_table$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.html"

$strain_table$tsv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.tsv.zip"

$strain_table$docx
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.docx"

$strain_table$csv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.csv"


$tox_table_ft
$tox_table_ft$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.html"

$tox_table_ft$tsv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.tsv.zip"

$tox_table_ft$docx
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.docx"

$tox_table_ft$csv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.csv"


$trait_table
$trait_table$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S3/table_S3.html"

$trait_table$tsv
[1] "

# Inputs

In [ ]:
# load H2 and h2 rdata files
load("data/processed/heritability/h2.list.mean.rda")

# load the H2 data from boots
load("data/processed/heritability/H2.parametric.boot.list.10000.rda")

# load the tox data
tox <- data.table::fread("data/processed/tox_data/con_metadata.csv") %>%
  dplyr::filter(type == "Toxicant")


# Main

In [ ]:
## H2 ##
# setup as data frame
H2.df.list <- NULL
for (i in 1:length(H2.list)) {
  drug <- names(H2.list)[i]
  o <- H2.list[[i]][[1]] %>%
    dplyr::mutate(drug = drug)

  H2.df.list[[i]] <- o
}
# bind list
H2.df <- data.table::rbindlist(H2.df.list) %>%
  dplyr::filter(drug %in% unique(tox$drug))

## h2 ##

h2.df.list.b <- NULL
for (i in 1:length(h2.list.mean)) {
  drug <- names(h2.list.mean)[i]
  o <- h2.list.mean[[i]] %>%
    dplyr::mutate(drug = drug)

  h2.df.list.b[[i]] <- o
}
# bind list
h2.df.b <- data.table::rbindlist(h2.df.list.b)

h.b2 <- dplyr::left_join(H2.df, h2.df.b) %>%
  dplyr::left_join(tox) %>%
  # dplyr::filter(big_class != "Flame Retardant") %>%
  dplyr::mutate(nice_drug_label2 = stringr::str_replace(nice_drug_label2, pattern = "uM", replacement = "µM"))


Joining with `by = join_by(drug)`
Joining with `by = join_by(drug)`

In [ ]:
# Clean up the summary data for the supplement
h2_summary_clean <- h.b2 %>%
  dplyr::select(
    nice_drug_label2,
    big_class,
    H2.point,
    H2.CI.lower,
    H2.CI.upper,
    H2.CI.level,
    h2.point,
    h2.lower,
    h2.upper
  ) %>%
  rename(
    "Toxicant" = nice_drug_label2,
    "Class" = big_class,
    "Broad-sense heritability" = H2.point,
    "Broad-sense heritability CI lower" = H2.CI.lower,
    "Broad-sense heritability CI upper" = H2.CI.upper,
    "Broad-sense heritability CI level" = H2.CI.level,
    "Narrow-sense heritability" = h2.point,
    "Narrow-sense heritability CI lower" = h2.lower,
    "Narrow-sense heritability CI upper" = h2.upper
  ) %>%
  dplyr::group_by(Class) %>%
  dplyr::arrange(Class, `Broad-sense heritability`)


In [ ]:
# Calculate group means
group_means <- h2_summary_clean %>%
  dplyr::summarise(
    Toxicant = "Group Mean",
    `Broad-sense heritability` = mean(`Broad-sense heritability`),
    `Broad-sense heritability CI lower` = mean(`Broad-sense heritability CI lower`),
    `Broad-sense heritability CI upper` = mean(`Broad-sense heritability CI upper`),
    `Narrow-sense heritability` = mean(`Narrow-sense heritability`),
    `Narrow-sense heritability CI lower` = mean(`Narrow-sense heritability CI lower`),
    `Narrow-sense heritability CI upper` = mean(`Narrow-sense heritability CI upper`)
  )

# Combine individual and group mean data
h2_summary_with_means <- dplyr::bind_rows(
  h2_summary_clean,
  group_means
) %>%
  dplyr::arrange(Class, Toxicant == "Group Mean", `Broad-sense heritability`)

# Create grouped data by Class
grouped_data <- as_grouped_data(
  x = h2_summary_with_means,
  groups = "Class"
)


In [ ]:
# Create flextable from grouped data
heritability_ft <- as_flextable(
  grouped_data,
  col_keys = c(
    "Toxicant",
    "Broad-sense heritability",
    "Narrow-sense heritability"
  ),
  hide_grouplabel = TRUE
) %>%
  flextable::set_header_labels(
    values = list(
      "Toxicant" = "Toxicant",
      "Broad-sense heritability" = "Broad-sense heritability (H²)",
      "Narrow-sense heritability" = "Narrow-sense heritability (h²)"
    )
  ) %>%
  # Combine heritability values with their CIs
  flextable::compose(
    j = "Broad-sense heritability",
    i = ~ !is.na(Toxicant),
    value = as_paragraph(
      sprintf("%.3f", `Broad-sense heritability`),
      " [",
      sprintf("%.3f", `Broad-sense heritability CI lower`),
      ", ",
      sprintf("%.3f", `Broad-sense heritability CI upper`),
      "]"
    )
  ) %>%
  flextable::compose(
    j = "Narrow-sense heritability",
    i = ~ !is.na(Toxicant),
    value = as_paragraph(
      sprintf("%.3f", `Narrow-sense heritability`),
      " [",
      sprintf("%.3f", `Narrow-sense heritability CI lower`),
      ", ",
      sprintf("%.3f", `Narrow-sense heritability CI upper`),
      "]"
    )
  ) %>%
  # make only the groups and header bold
  flextable::bold(
    j = 1,
    i = ~ is.na(Toxicant) | Toxicant == "Group Mean",
    bold = TRUE,
    part = "body"
  ) %>%
  flextable::bold(part = "header") %>%
  flextable::line_spacing(space = 1, part = "all") %>%
  flextable::align(j = 1, align = "left") %>%
  flextable::align(j = 2:3, align = "center") %>%
  flextable::hline(
    i = ~ is.na(Toxicant) | Toxicant == "Group Mean",
    part = "body"
  ) %>%
  flextable::set_table_properties(
    layout = "autofit",
    width = 1,
    opts_word = list(
      split = FALSE,
      keep_with_next = TRUE
    )
  )
# Save the table using standardized functions
save_supp_table_csv(h2_summary_with_means, "heritability_summary")
save_supp_table_flextable(heritability_ft, "heritability_summary")


In [ ]:
heritability_ft


Toxicant Broad-sense heritability (H²) Narrow-sense heritability (h²) Flame Retardant TPhP 6.25 µM 0.470 [ 0.404 , 0.527 ] 0.117 [ 0.046 , 0.188 ] TPhP 50 µM 0.660 [ 0.604 , 0.706 ] 0.337 [ 0.239 , 0.435 ] Group Mean 0.565 [ 0.504 , 0.616 ] 0.227 [ 0.143 , 0.311 ] Fungicide Mancozeb 0.318 [ 0.262 , 0.371 ] 0.163 [ 0.088 , 0.238 ] Chlorothalonil 0.432 [ 0.371 , 0.487 ] 0.165 [ 0.090 , 0.240 ] Carboxin 0.474 [ 0.413 , 0.529 ] 0.076 [ 0.020 , 0.131 ] Pyraclostrobin 0.555 [ 0.494 , 0.607 ] 0.274 [ 0.183 , 0.365 ] Group Mean 0.445 [ 0.385 , 0.499 ] 0.169 [ 0.095 , 0.244 ] Herbicide 2,4-D 0.300 [ 0.245 , 0.356 ] 0.036 [ -0.007 , 0.079 ] Atrazine 0.470 [ 0.409 , 0.527 ] 0.041 [ -0.003 , 0.086 ] Paraquat 62.5 µM 0.622 [ 0.558 , 0.676 ] 0.085 [ 0.017 , 0.152 ] Paraquat 250 µM 0.738 [ 0.692 , 0.776 ] 0.507 [ 0.404 , 0.609 ] Group Mean 0.533 [ 0.476 , 0.584 ] 0.167 [ 0.103 , 0.232 ] Insecticide Propoxur 0.286 [ 0.232 , 0.338 ] 0.104 [ 0.042 , 0.167 ] Malathion 0.416 [ 0.353 , 0.475 ] 0.214 [ 0.131 , 0.298 ] Carbaryl 0.493 [ 0.431 , 0.548 ] 0.137 [ 0.067 , 0.207 ] Chlorpyrifos 0.498 [ 0.432 , 0.555 ] 0.234 [ 0.140 , 0.327 ] Methomyl 0.677 [ 0.615 , 0.729 ] 0.434 [ 0.307 , 0.561 ] Aldicarb 0.690 [ 0.640 , 0.732 ] 0.174 [ 0.097 , 0.251 ] Chlorfenapyr 0.770 [ 0.720 , 0.809 ] 0.238 [ 0.135 , 0.341 ] Group Mean 0.547 [ 0.489 , 0.598 ] 0.219 [ 0.131 , 0.308 ] Metal Silver 7.8 µM 0.327 [ 0.270 , 0.382 ] 0.071 [ 0.017 , 0.125 ] Lead 0.334 [ 0.277 , 0.387 ] 0.266 [ 0.175 , 0.356 ] Zinc 0.363 [ 0.302 , 0.420 ] 0.081 [ 0.019 , 0.142 ] Arsenic 0.442 [ 0.380 , 0.498 ] 0.066 [ 0.013 , 0.119 ] Mercury 0.601 [ 0.543 , 0.652 ] 0.198 [ 0.117 , 0.280 ] Copper 0.616 [ 0.558 , 0.666 ] 0.371 [ 0.272 , 0.470 ] Cadmium 0.636 [ 0.580 , 0.684 ] 0.307 [ 0.211 , 0.402 ] Nickel 0.700 [ 0.650 , 0.742 ] 0.334 [ 0.237 , 0.431 ] Silver 250 µM 0.820 [ 0.783 , 0.850 ] 0.440 [ 0.335 , 0.544 ] Group Mean 0.538 [ 0.483 , 0.587 ] 0.237 [ 0.155 , 0.319 ]